In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

#import numpy as np # linear algebra
#import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

#import os
#for dirname, _, filenames in os.walk('/kaggle/input'):
#    for filename in filenames:
#        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [4]:
#|default_exp app

In [ ]:
#|export

import warnings
warnings.filterwarnings('ignore')

In [ ]:
#|export

import pathlib
temp = pathlib.PosixPath
pathlib.PosixPath = pathlib.WindowsPath

In [ ]:
#|export

import fastbook
import pandas as pd
import numpy as np
import os 

from fastbook import *
from fastai.vision.widgets import *
import gradio as gr

In [ ]:
im = PILImage.create('plume1.png')
im.thumbnail((182,192))
im

In [ ]:
path = Path('/kaggle/input/speuntapped/train')

In [ ]:
fns = get_image_files(path)
fns

In [ ]:
plume = DataBlock(
    blocks=(ImageBlock, CategoryBlock),
    get_items=get_image_files,
    splitter=RandomSplitter(valid_pct=0.2,seed=42),
    get_y=parent_label
)

In [ ]:
plume = plume.new(
    item_tfms=RandomResizedCrop(152, min_scale=0.5),
    batch_tfms=aug_transforms()
)

In [ ]:
dls = plume.dataloaders(path)

In [ ]:
dls.valid.show_batch(max_n=12, nrows=4) 

In [ ]:
learn = vision_learner(dls, resnet50, metrics=error_rate)
learn.fine_tune(epochs=3)

In [ ]:
from IPython.core.pylabtools import figsize 
interp = ClassificationInterpretation.from_learner(learn)
interp.plot_confusion_matrix(figsize=(10,10))

In [ ]:
interp.plot_top_losses(20, nrows=10, figsize=(20,20))

In [ ]:
test_image = '/kaggle/input/speuntapped/test/test/ang20190922t192642-4_r4928_c373-plume.png'

In [ ]:
is_plume,_,probs = learn.predict(PILImage.create(test_image))
print(f"This is a: {is_plume}.")
print(f"Probability it's a plume: {probs[1]:.4f}")

In [ ]:
learn.export('methane plume detect.pkl')

In [ ]:
learn = load_learner('methane plume detect.pkl')

In [ ]:
learn.predict(im)

In [ ]:
#|export

categories = ('No Plume', 'Plume')

def classify_image(img):
    pred, idx, probs = learn.predict(img)
    return dict(zip(categories, map(float, probs)))

In [ ]:
classify_image(im)

In [ ]:
#|export

image = gr.inputs.Image(shape = (152, 152))
label = gr.outputs.Label()
examples = ['plume1.png', 'plume2.png', 'noplume1.png', 'noplume2.png']

intf = gr.Interface(fn = classify_image, inputs = image, outputs = label, examples = examples)
intf.launch(inline = False)

In [7]:
import nbdev
nbdev.export.nb_export('methane-plume-classification-yes-or-no.ipynb')
print('Export successful')

Export successful
